In [1]:
from file_processing import *

In [2]:
import json
import pandas as pd
import numpy as np

In [4]:
!mkdir 'dummy_data/1'
!mkdir 'dummy_data/2'
!mkdir 'dummy_data/4'

mkdir: cannot create directory ‘dummy_data/1’: File exists
mkdir: cannot create directory ‘dummy_data/2’: File exists
mkdir: cannot create directory ‘dummy_data/4’: File exists


In [3]:
dummy_data_path = 'dummy_data/'

In [4]:
d = {
    'route_id': [],
    'user_id': [],
    'model_version': [],
    'path': [],
    'created_at': [],
    'descriptors': []
}

In [5]:
def gen_descriptor(detector, category, img_name, dataset_path):
    img = cv2.imread(os.path.join(dataset_path, category, img_name))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, des = detector.detectAndCompute(img, None)
    return des

In [6]:
orb = cv2.ORB_create(450)

The Castle images - "standard gym"

In [7]:
castle_data_path = 'data/categories_castle_cafe/categories_blurred_photos_removed_small'

In [8]:
list_of_cats = listdir_no_hidden(castle_data_path)
list_of_cats.sort(key=int)
list_of_cats = [x for x in list_of_cats if x not in ['47','48','49','50']] # don't want all the photos from manual test

In [9]:
for c in list_of_cats:
    category_path = os.path.join(castle_data_path, f"{c}/")
    list_of_filenames = get_filenames(category_path)
    for f in list_of_filenames[:4]:  # up to four images, otherwise the file will be very big
        d['route_id'].append(c)
        if int(c)<10:
            d['user_id'].append(1)
            d['created_at'].append("2020-02-15 09:00:00+0")
            d['path'].append(f"s3://climbicus-dev/route_images/dummy_data/gym_id=1/year=2020/month=02/{c}_{f}")
        elif int(c)<30:
            d['user_id'].append(2)
            d['created_at'].append("2020-02-10 12:00:00+0")
            d['path'].append(f"s3://climbicus-dev/route_images/dummy_data/gym_id=1/year=2020/month=02/{c}_{f}")
        else:
            d['user_id'].append(3)
            d['created_at'].append("2020-02-26 23:00:00+0")
            d['path'].append(f"s3://climbicus-dev/route_images/dummy_data/gym_id=1/year=2020/month=02/{c}_{f}")
        d['model_version'].append('dummy_version')
        des = gen_descriptor(orb, c, f, castle_data_path)
        des_json = json.dumps(des.tolist())
        d['descriptors'].append(des_json)
        shutil.copyfile(category_path+f, f"{dummy_data_path}/1/{c}_{f}")

Yonder images - 300, but don't need to be real, I will duplicate the images that I have.

Due to the requirement to have a unique image path, I will add a string 'dup' to the name

In [10]:
yonder_data_path = 'data/categories_yonder_cbir_small'

In [11]:
list_of_cats = listdir_no_hidden(yonder_data_path)
list_of_cats.sort(key=int)
print(len(list_of_cats))

38


In [12]:
route_counter = 0
loop_counter = 0
while route_counter < 300:
    for c in list_of_cats:
        category_path = os.path.join(yonder_data_path, f"{c}/")
        list_of_filenames = get_filenames(category_path)
        for f in list_of_filenames[:2]: # just take two images as otherwise it's crazy
            if route_counter<300:  # otherwise it finishes the whole loop
                d['route_id'].append(f"{int(c) + 46 + loop_counter*38}")  # to account for the initial 46 castle routes and duplicates
                if int(c)<10:
                    d['user_id'].append(1)
                    d['created_at'].append("2020-02-14 09:00:00+0")
                    d['path'].append(f"s3://climbicus-dev/route_images/dummy_data/gym_id=2/year=2020/month=02/{c}_{route_counter}_{f}")
                elif int(c)<30:
                    d['user_id'].append(2)
                    d['created_at'].append("2020-02-19 12:00:00+0")
                    d['path'].append(f"s3://climbicus-dev/route_images/dummy_data/gym_id=2/year=2020/month=02/{c}_{route_counter}_{f}")
                else:
                    d['user_id'].append(3)
                    d['created_at'].append("2020-02-03 23:00:00+0")
                    d['path'].append(f"s3://climbicus-dev/route_images/dummy_data/gym_id=2/year=2020/month=02/{c}_{route_counter}_{f}")
                d['model_version'].append('dummy_version')
                des = gen_descriptor(orb, c, f, yonder_data_path)
                des_json = json.dumps(des.tolist())
                d['descriptors'].append(des_json)
                shutil.copyfile(category_path+f, f"{dummy_data_path}/2/{c}_{route_counter}_{f}")
            else:
                break
        route_counter += 1
    loop_counter += 1

Gym 4  - some strange images

I've pre-prepared a dataset_path of these weird images

In [13]:
odd_data_path = 'data/edge_case_routes/'

In [14]:
list_of_cats = listdir_no_hidden(odd_data_path)
list_of_cats.sort(key=int)
print(len(list_of_cats))

3


In [15]:
for c in list_of_cats:
    category_path = os.path.join(odd_data_path, f"{c}/")
    list_of_filenames = get_filenames(category_path)
    for f in list_of_filenames[:4]:  # up to four images, otherwise the file will be very big
        d['route_id'].append(f"{int(c) + 46 + 300}")
        if int(c)<10:
            d['user_id'].append(1)
            d['created_at'].append("2020-02-15 09:00:00+0")
            d['path'].append(f"s3://climbicus-dev/route_images/dummy_data/gym_id=4/year=2020/month=02/{int(c) + 46 + 300}_{f}")
        elif int(c)<30:
            d['user_id'].append(2)
            d['created_at'].append("2020-02-10 12:00:00+0")
            d['path'].append(f"s3://climbicus-dev/route_images/dummy_data/gym_id=4/year=2020/month=02/{int(c) + 46 + 300}_{f}")
        else:
            d['user_id'].append(3)
            d['created_at'].append("2020-02-26 23:00:00+0")
            d['path'].append(f"s3://climbicus-dev/route_images/dummy_data/gym_id=4/year=2020/month=02/{int(c) + 46 + 300}_{f}")
        d['model_version'].append('dummy_version')
        des = gen_descriptor(orb, c, f, odd_data_path)
        try:
            des_json = json.dumps(des.tolist())
        except AttributeError:
            des_json = json.dumps(np.zeros((10, 32)).tolist())
        d['descriptors'].append(des_json)
        shutil.copyfile(category_path+f, f"{dummy_data_path}/4/{int(c) + 46 + 300}_{f}")

In [16]:
df = pd.DataFrame.from_dict(d)

In [17]:
df.shape

(781, 6)

In [18]:
# 46 + 300 + 3 = 349
df.route_id.nunique()

349

In [19]:
df.head()

,route_id,user_id,model_version,path,created_at,descriptors
0,1,1,dummy_version,s3://climbicus-dev/route_images/dummy_data/gym...,2020-02-15 09:00:00+0,"[[18, 203, 70, 156, 113, 144, 184, 230, 157, 4..."
1,1,1,dummy_version,s3://climbicus-dev/route_images/dummy_data/gym...,2020-02-15 09:00:00+0,"[[50, 155, 75, 188, 116, 195, 90, 119, 133, 76..."
2,1,1,dummy_version,s3://climbicus-dev/route_images/dummy_data/gym...,2020-02-15 09:00:00+0,"[[231, 133, 204, 158, 136, 36, 78, 174, 147, 1..."
3,1,1,dummy_version,s3://climbicus-dev/route_images/dummy_data/gym...,2020-02-15 09:00:00+0,"[[95, 21, 35, 138, 173, 244, 75, 48, 37, 96, 1..."
4,2,1,dummy_version,s3://climbicus-dev/route_images/dummy_data/gym...,2020-02-15 09:00:00+0,"[[33, 232, 218, 86, 0, 113, 23, 99, 133, 6, 23..."


In [29]:
df.iloc[-1].path

's3://climbicus-dev/route_images/dummy_data/gym_id=4/year=2020/month=02/349_IMG_20191215_092551.jpg'

In [22]:
df.to_csv('new_route_images.csv', index=False)

#### Generating images to upload to S3 from route_images.csv with imagefile

In [139]:
route_images = pd.read_csv('route_images_descriptors.csv')

In [140]:
route_images['imagefilename'] = route_images.path.apply(lambda x: x.split('/')[8].replace('_IMG', '/IMG'))

In [ ]:
import shutil, os
dest = f"data/upload/"
if not os.path.exists(dest):
    os.mkdir(dest)
origin = 'data/categories_castle_cafe/categories_blurred_photos_removed_small/'
for _, row in route_images.iterrows():
    filepath = row['imagefilename']
    category = filepath.split('/')[0]
    filename = filepath.split('/')[1]
    shutil.copyfile(os.path.join(origin, filepath), 
                    os.path.join(dest, f'{category}_{filename}'))